In [147]:
import pygmsh
import numpy as np


with pygmsh.geo.Geometry() as geom:
    poly = geom.add_polygon(
        [
            [0.0, 0.0],
            [1.0, -0.2],
            [1.1, 1.2],
            [0.1, 0.7],
        ],
        mesh_size=0.1,
    )
    geom.extrude(poly, [0.0, 0.3, 1.0], num_layers=5)
    mesh = geom.generate_mesh()

C:\Users\Wes\AppData\Roaming\Python\Python37\site-packages\numpy\ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


In [148]:
import k3d


plot = k3d.plot()
plt_mesh = k3d.mesh(mesh.points.astype(np.float32),
                    mesh.cells_dict['triangle'].astype(np.uint32),
                    wireframe=True, color=0x000000)
plot += plt_mesh
plot.display()

Output()

In [242]:
with pygmsh.occ.Geometry() as geom:
    geom.characteristic_length_min = 0.1
    geom.characteristic_length_max = 0.1

#     rectangle = geom.add_rectangle([-1.0, -1.0, 0.0], 2.0, 2.0)
#     disk1 = geom.add_disk([-1.2, 0.0, 0.0], 0.5)
#     disk2 = geom.add_disk([+1.2, 0.0, 0.0], 0.5)

#     disk3 = geom.add_disk([0.0, -0.9, 0.0], 0.5)
#     disk4 = geom.add_disk([0.0, +0.9, 0.0], 0.5)
#     flat = geom.boolean_difference(
#         geom.boolean_union([rectangle, disk1, disk2]),
#         geom.boolean_union([disk3, disk4]),
#     )
    
    rectangle = geom.add_rectangle([-1.0, -1.0, 0.0], 2.0, 2.0)
    _, base, _ = geom.extrude(rectangle, [0, 0, 0.3])
    
    square = geom.add_rectangle([-0.5, -0.5, 0.3], 0.5, 0.5)
    _, pillar, _ = geom.extrude(square, [0, 0, 1.0])
    
    disk = geom.add_disk([-0.25, -0.25, 1.3], 0.5)
    _, top, _ = geom.extrude(disk, [0, 0, 0.1])
    
    geom.boolean_union([base, pillar, top])

    mesh = geom.generate_mesh()
    
plot = k3d.plot()
plt_mesh = k3d.mesh(mesh.points.astype(np.float32),
                    mesh.cells_dict['triangle'].astype(np.uint32),
                    wireframe=True, color=0x000000)
plot += plt_mesh
plot.display()

C:\Users\Wes\AppData\Roaming\Python\Python37\site-packages\numpy\ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


Output()

In [223]:
with pygmsh.geo.Geometry() as geom:
    poly = geom.add_polygon(
        [
            [0.0, 0.0, 0.0],
            [1.0, -0.2, 0.0],
            [1.1, 1.2, 0.0],
            [0.1, 0.7, 0.0],
        ],
        mesh_size=0.1,
    )
    _, a, _ = geom.extrude(poly, [0.0, 0.3, 1.0], num_layers=5)
    poly = geom.add_polygon(
        [
            [0.0, 0.3, 1.0],
            [1.0, 0.1, 1.0],
            [1.1, 1.5, 1.0],
            [0.1, 1.0, 1.0],
        ],
        mesh_size=0.1,
    )
    _, b, _ = geom.extrude(poly, [0.0, -0.3, 1.0], num_layers=5)
    mesh = geom.generate_mesh()
    
plot = k3d.plot()
plt_mesh = k3d.mesh(mesh.points.astype(np.float32),
                    mesh.cells_dict['triangle'].astype(np.uint32),
                    wireframe=True, color=0x000000)
plot += plt_mesh
plot.display()

C:\Users\Wes\AppData\Roaming\Python\Python37\site-packages\numpy\ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


Output()

In [243]:
mesh.write('out.vtk')

In [244]:
from sfepy.discrete import fem


sfe_mesh = fem.Mesh.from_file('out.vtk')
data = list(sfe_mesh._get_io_data(cell_dim_only=[3]))  # strip non-3d elements
sfe_mesh = fem.Mesh.from_data(sfe_mesh.name, *data)
domain = fem.FEDomain('domain', sfe_mesh)

sfepy: reading mesh (out.vtk)...
sfepy:   warning: unknown cell type vertex with dimension 3
sfepy:   number of vertices: 2340
sfepy:   number of cells:
sfepy:     1_2: 317
sfepy:     2_3: 3498
sfepy:     3_4: 8735
sfepy: ...done in 0.01 s


In [245]:
from sfepy.mechanics import matcoefs
from sfepy.discrete import Material


# these are for stainless steel 316L
m = Material('m', D=matcoefs.stiffness_from_youngpoisson(dim=3, young=1.93e9, poisson=0.275), rho=8000.0)

In [246]:
from sfepy.discrete import conditions


min_z, max_z = domain.get_mesh_bounding_box()[:, 2]
epsilon = 1e-2
omega = domain.create_region('Omega', 'all')
bot_region = domain.create_region('bot', 'vertices in z < %.10f' % (min_z + epsilon), 'vertex')
top_region = domain.create_region('top', 'vertices in z > %.10f' % (max_z - epsilon), 'vertex')

z_displacement = -0.5
displace_top = conditions.EssentialBC('displace_top', top_region, {'u.[0,1]': 0.0, 'u.[2]': z_displacement})
fix_bot = conditions.EssentialBC('fix_bot', bot_region, {'u.all': 0.0})

In [247]:
from sfepy.discrete import FieldVariable


field = fem.Field.from_args('fu', np.float64, 'vector', omega, approx_order=1)
u = FieldVariable('u', 'unknown', field)
v = FieldVariable('v', 'test', field, primary_var_name='u')

In [248]:
from sfepy.discrete import Equation, Equations, Integral, Problem
from sfepy.terms import Term


integral = Integral('i', order=1)
t1 = Term.new('dw_lin_elastic(m.D, v, u)', integral, omega, m=m, v=v, u=u)
eq1 = Equation('balance_of_forces', t1)
eqs = Equations([eq1])

pb = Problem('elasticity', equations=eqs)
pb.save_regions_as_groups('regions')
pb.set_bcs(ebcs=conditions.Conditions([fix_bot, displace_top]))

sfepy: saving regions as groups...
sfepy:   Omega
sfepy:   bot
sfepy:   top
sfepy: ...done


In [249]:
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton
from sfepy.discrete import Problem


ls = ScipyDirect({})
nls = Newton({}, lin_solver=ls)
pb.set_solver(nls)

In [250]:
state = pb.solve()

sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 0.00 s
sfepy: matrix shape: (5106, 5106)
sfepy: assembling matrix graph...
sfepy: ...done in 0.04 s
sfepy: matrix structural nonzeros: 174186 (6.68e-03% fill)
sfepy: updating variables...
sfepy: ...done
sfepy: updating materials...
sfepy:     m
sfepy: ...done in 0.00 s
sfepy: nls: iter: 0, residual: 1.148695e+09 (rel: 1.000000e+00)
sfepy:   residual:    0.00 [s]
sfepy:     matrix:    0.04 [s]
sfepy:      solve:    0.09 [s]
sfepy: warning: linear system solution precision is lower
sfepy: then the value set in solver options! (err = 1.932731e-06 < 1.000000e-10)
sfepy: nls: iter: 1, residual: 1.842687e-06 (rel: 1.604158e-15)
sfepy: solved in 1 steps in 0.15 seconds


In [251]:
from sfepy.mechanics.tensors import get_von_mises_stress


disp = np.array(u.data[0]).reshape(-1, 3)
strain = pb.evaluate('ev_cauchy_strain.2.Omega(u)', u=u, mode='el_avg')
stress = pb.evaluate('ev_cauchy_stress.2.Omega(m.D, u)', m=m, u=u, mode='el_avg')
vms = get_von_mises_stress(stress.squeeze())

sfepy: equation "tmp":
sfepy: ev_cauchy_strain.2.Omega(u)
sfepy: updating materials...
sfepy: ...done in 0.00 s
sfepy: equation "tmp":
sfepy: ev_cauchy_stress.2.Omega(m.D, u)
sfepy: updating materials...
sfepy:     m
sfepy: ...done in 0.01 s


In [252]:
node_vms = np.zeros(omega.vertices.shape)
node_vms[sfe_mesh.get_conn('3_4')] += vms

In [255]:
plot = k3d.plot()
plt_mesh = k3d.mesh(sfe_mesh.coors.astype(np.float32) + disp.astype(np.float32),
                    sfe_mesh.get_conn('3_4').astype(np.uint32),
                    wireframe=True, color=0x000000,
                    color_map = k3d.colormaps.basic_color_maps.Jet,
                    attribute= node_vms,
                    color_range = [0, node_vms.max()])
plot += plt_mesh
plot.display()

Output()